In [2]:
pip install pandas_profiling

  Using cached pandas_profiling-2.8.0-py2.py3-none-any.whl (259 kB)
  Using cached matplotlib-3.2.2-cp37-cp37m-win_amd64.whl (9.2 MB)
  Using cached visions-0.4.4-py3-none-any.whl (59 kB)
  Using cached phik-0.10.0-py3-none-any.whl (599 kB)
  Using cached missingno-0.4.2-py3-none-any.whl (9.7 kB)
  Using cached tangled_up_in_unicode-0.0.6-py3-none-any.whl (3.1 MB)
Processing c:\users\dell\appdata\local\pip\cache\wheels\70\e1\52\5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655\htmlmin-0.1.12-py3-none-any.whl
Processing c:\users\dell\appdata\local\pip\cache\wheels\16\72\b0\e5be34699908d9ff25dcb3debbb717987b766af61bbddffdfe\imagehash-4.1.0-py2.py3-none-any.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.42.1
    Uninstalling tqdm-4.42.1:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tqdm-4.42.1.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.



In [1]:
#necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import plotly.offline as po
import plotly.graph_objs as go
%matplotlib inline

ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
#perform EDA in just one line of code
#pandas_profiling.ProfileReport(pd.read_csv('churn.csv'))

In [ ]:
churn_dataset = pd.read_csv('churn.csv')

In [ ]:
churn_dataset.shape

In [ ]:
churn_dataset.loc[churn_dataset.Churn == 'No','Churn'] = 0
churn_dataset.loc[churn_dataset.Churn == 'Yes','Churn'] = 1

In [ ]:
churn_dataset.head()

In [7]:
churn_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:

#Replace null value
churn_dataset['TotalCharges'] = churn_dataset["TotalCharges"].replace(r'\s+', np.nan, regex=True)
churn_dataset['Churn'] = churn_dataset["Churn"].replace(r'\s+', np.nan, regex=True)
#Drop null value
churn_dataset = churn_dataset[churn_dataset["TotalCharges"].notnull()]
churn_dataset = churn_dataset.reset_index()[churn_dataset.columns]

churn_dataset = churn_dataset[churn_dataset["Churn"].notnull()]
churn_dataset = churn_dataset.reset_index()[churn_dataset.columns]

#convert TotalCharges column value to float
#churn_dataset["TotalCharges"] = churn_dataset["TotalCharges#].astype(float)
churn_dataset['TotalCharges'] = pd.to_numeric(churn_dataset["TotalCharges"])
churn_dataset['Churn'] = pd.to_numeric(churn_dataset["Churn"])

"""
# We need to convert the Total Charges from object type to Numeric
df['TotalCharges'] = df['TotalCharges'].replace(r'\s+', np.nan, regex=True)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

"""

"\n# We need to convert the Total Charges from object type to Numeric\ndf['TotalCharges'] = df['TotalCharges'].replace(r'\\s+', np.nan, regex=True)\ndf['TotalCharges'] = pd.to_numeric(df['TotalCharges'])\n\n"

In [9]:
churn_dataset["Churn"].value_counts().values

array([5163, 1869], dtype=int64)

In [10]:
#visualize total custoner churn
"""
plot_by_churn_label = churn_dataset["Churn"].value_counts().keys().tolist()
plot_by_churn_value = churn_dataset["Churn"].value_counts().values.tolist()

plot_data = [
    go.Pie(labels = plot_by_churn_label,
          values = plot_by_churn_value,
          marker = dict(colors = ['Teal','Grey'],
                       line = dict(color = "white",
                                  width = 1.5)),
          rotation= 90,
          hoverinfo="label+value+text",
          hole=.6)
]
plot_layout = go.Layout(dict(title = "Customer Churn",
                            plot_bgcolor = "rgb(243,243,243)",
                             paper_bgcolor = "rgb(243,243,243)",))

fig = go.Figure(data=plot_data,layout=plot_layout)
po.iplot(fig)
"""

'\nplot_by_churn_label = churn_dataset["Churn"].value_counts().keys().tolist()\nplot_by_churn_value = churn_dataset["Churn"].value_counts().values.tolist()\n\nplot_data = [\n    go.Pie(labels = plot_by_churn_label,\n          values = plot_by_churn_value,\n          marker = dict(colors = [\'Teal\',\'Grey\'],\n                       line = dict(color = "white",\n                                  width = 1.5)),\n          rotation= 90,\n          hoverinfo="label+value+text",\n          hole=.6)\n]\nplot_layout = go.Layout(dict(title = "Customer Churn",\n                            plot_bgcolor = "rgb(243,243,243)",\n                             paper_bgcolor = "rgb(243,243,243)",))\n\nfig = go.Figure(data=plot_data,layout=plot_layout)\npo.iplot(fig)\n'

In [11]:
#One hot encoding

churn_dataset = pd.get_dummies(churn_dataset,columns = ['Contract','Dependents','DeviceProtection','gender','InternetService','MultipleLines','OnlineBackup', 'OnlineSecurity'
                                                        ,'PaperlessBilling','Partner','PaymentMethod','PhoneService', 'SeniorCitizen', 'StreamingMovies','StreamingTV','TechSupport'],
                              
                              drop_first = True)
                              


In [12]:
#scale the value bring there in to same range
from sklearn.preprocessing import StandardScaler

sd = StandardScaler()
column_fit = ['tenure','MonthlyCharges','TotalCharges']
churn_dataset[column_fit] = sd.fit_transform(churn_dataset[column_fit])

In [13]:
churn_dataset.head()

,customerID,tenure,MonthlyCharges,TotalCharges,Churn,Contract_One year,Contract_Two year,Dependents_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PhoneService_Yes,SeniorCitizen_1,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No internet service,StreamingTV_Yes,TechSupport_No internet service,TechSupport_Yes
0,7590-VHVEG,-1.280248,-1.161694,-0.994194,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,5575-GNVDE,0.064303,-0.260878,-0.173740,0,1,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
2,3668-QPYBK,-1.239504,-0.363923,-0.959649,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,7795-CFOCW,0.512486,-0.747850,-0.195248,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,9237-HQITU,-1.239504,0.196178,-0.940457,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [14]:
y = churn_dataset['Churn']
X = churn_dataset.drop(['Churn','customerID'],axis = 1)

In [15]:
#split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.30, random_state = 50)

In [16]:
#churn_dataset.columns

Index(['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'Contract_One year', 'Contract_Two year', 'Dependents_Yes',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'gender_Male', 'InternetService_Fiber optic', 'InternetService_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'PaperlessBilling_Yes', 'Partner_Yes',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'PhoneService_Yes', 'SeniorCitizen_1',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'TechSupport_No internet service', 'TechSupport_Yes'],
      dtype='object')

In [31]:
#model
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train,y_train)

pred =clf.predict_proba(X_test)




In [18]:
churn_dataset['prob'] = clf.predict_proba(churn_dataset[X_test.columns])[:,1]

In [19]:
churn_dataset[['customerID','prob']].head(10)

,customerID,prob
0,7590-VHVEG,0.649601
1,5575-GNVDE,0.043494
2,3668-QPYBK,0.343624
3,7795-CFOCW,0.025990
4,9237-HQITU,0.693996
5,9305-CDSKC,0.781756
6,1452-KIOVK,0.491846
7,6713-OKOMC,0.291489
8,7892-POOKP,0.593713
9,6388-TABGU,0.011926


In [20]:
churn_dataset[['customerID','prob']].to_csv("solution.csv", sep=',')